# 复制并修改这个文件即可

In [ ]:
# ------------------------------------------------------------
# PPQ 最佳实践示例工程，在这个工程中，我们将向你展示如何充分调动 PPQ 的各项功能
# ------------------------------------------------------------
import torch
from ppq import *
from ppq.api import *
import torchvision
# ------------------------------------------------------------
# Step - 1. 加载校准集与模型
# ------------------------------------------------------------
BATCHSIZE   = 32
INPUT_SHAPE = [BATCHSIZE, 3, 224, 224]
DEVICE      = 'cuda'
PLATFORM    = TargetPlatform.TRT_INT8

CALIBRATION = [torch.rand(size=INPUT_SHAPE) for _ in range(32)]
QS          = QuantizationSettingFactory.default_setting()
def collate_fn(batch: torch.Tensor) -> torch.Tensor:
    return batch.to(DEVICE)

In [ ]:
model = torchvision.models.mobilenet.mobilenet_v2(pretrained=True)
model = model.to(DEVICE)

In [ ]:
# ------------------------------------------------------------
# Step - 2. 执行首次量化，完成逐层误差分析
# ------------------------------------------------------------
with ENABLE_CUDA_KERNEL():
    quantized = quantize_torch_model(
        model=model, calib_dataloader=CALIBRATION,
        calib_steps=32, input_shape=INPUT_SHAPE,
        collate_fn=collate_fn, platform=PLATFORM, setting=QS,
        onnx_export_file='Output/onnx.model', device=DEVICE, verbose=0)
    
    
    #三个分析结果
    #这个是从下到上每层累计误差，所以会不断变大，这个是最终评估标准
#留意网络输出的误差情况，如果你想获得一个精度较高的量化网络，那么那些靠近输出的节点误差不应超过 10%！！！！
    # 该方法只衡量 Conv, Gemm 算子的误差情况，如果你对其余算子的误差情况感兴趣，需要手动修改方法逻辑
    reports = graphwise_error_analyse(
        graph=quantized, running_device=DEVICE, collate_fn=collate_fn,
        dataloader=CALIBRATION)

    # ------------------------------------------------------------
    # layerwise_error_analyse 是更为强大的分析方法，它分析算子的量化敏感性
    # 与 graphwise_error_analyse 不同，该方法分析的误差不是累计的
    # 该方法首先解除网络中所有算子的量化，而后单独地量化每一个 Conv, Gemm 算子
    # 以此来衡量量化单独一个算子对网络输出的影响情况，该方法常被用来决定网络调度与混合精度量化
    # 你可以将那些误差较大的层送往 TargetPlatform.FP32
    # ------------------------------------------------------------
    reports = layerwise_error_analyse(
        graph=quantized, running_device=DEVICE, collate_fn=collate_fn,
        dataloader=CALIBRATION)

    # ------------------------------------------------------------
    # statistical_analyse 是强有力的统计分析方法，该方法统计每一层的输入、输出以及参数的统计分布情况
    # 使用这一方法，你将更清晰地了解网络的量化情况，并能够有针对性地选择优化方案
    # 推荐在网络量化情况不佳时，使用 statistical_analyse 辅助你的分析
    # 该方法不打印任何数据，你需要手动将数据保存到硬盘并进行分析
    # ------------------------------------------------------------
    report = statistical_analyse(
        graph=quantized, running_device=DEVICE, 
        collate_fn=collate_fn, dataloader=CALIBRATION)

    from pandas import DataFrame

    report = DataFrame(report)
    report.to_csv('1.csv')

In [ ]:
# ------------------------------------------------------------
# Step - 3. 根据逐层误差情况，将部分难以量化的层直接送到非量化平台
# 在这个例子中，我们解除前十个误差最大的层的量化，这只是一个示例
# 为了确保量化精度达到较高水准，通常只有个别层需要解除量化。
# 不要妄图单纯使用调度解决所有精度问题，调度体现了运行效率与网络精度之间的权衡
# 我们后续还可以通过调节校准算法与量化参数来提升精度
# ------------------------------------------------------------

# 从大到小排序单层误差
sensitivity = [(op_name, error) for op_name, error in reports.items()]
sensitivity = sorted(sensitivity, key=lambda x: x[1], reverse=True)

# 将前十个误差最大的层送上 FP32
for op_name, _ in sensitivity[: 10]:
    QS.dispatching_table.append(operation=op_name, platform=TargetPlatform.FP32)

In [ ]:
# ------------------------------------------------------------
# Step - 4. 选择一个合适的校准算法，最小化量化误差
# 这一过程需要你手动调整 QS 中的校准算法，需要反复执行和对比
# 在这里我们只推荐你更改激活值的校准算法，对于参数而言
# 在 INT8 的量化场景中，minmax 往往都是最好的校准策略
# 在这个场景下，不能使用 isotone 策略
# ------------------------------------------------------------
for calib_algo in {'minmax', 'percentile', 'kl', 'mse'}:
    QS.quantize_activation_setting.calib_algorithm = calib_algo

    with ENABLE_CUDA_KERNEL():
        quantized = quantize_torch_model(
            model=model, calib_dataloader=CALIBRATION,
            calib_steps=32, input_shape=INPUT_SHAPE,
            collate_fn=collate_fn, platform=PLATFORM, setting=QS,
            onnx_export_file='Output/onnx.model', device=DEVICE, verbose=0)

        print(f'Error Report of Algorithm {calib_algo}: ')
        reports = graphwise_error_analyse(
            graph=quantized, running_device=DEVICE, 
            collate_fn=collate_fn, dataloader=CALIBRATION)

# 在确定了一种校准算法之后，你还可以修改 ppq.core.common 中的相关属性来取得更优结果
# 下列参数将影响校准效果：
    # Observer 中 hist 箱子的个数
    # OBSERVER_KL_HIST_BINS = 4096
    # Observer 中 percentile 的参数
    # OBSERVER_PERCENTILE = 0.9999
    # Observer 中 mse 校准方法 hist 箱子的个数
    # OBSERVER_MSE_HIST_BINS = 2048
    # Observer 中 mse 计算的间隔，间隔越小，所需时间越长
    # OBSERVER_MSE_COMPUTE_INTERVAL = 8

# 在完成测试后，选择一种最为合适的校准算法，我们此处以 percentile 为例
QS.quantize_activation_setting.calib_algorithm = 'percentile'

In [ ]:
# ------------------------------------------------------------
# Step - 5. 再次检查我们的量化误差，如果与预期仍有差距
# 则我们可以进一步使用优化过程来调节网络参数
# ------------------------------------------------------------
with ENABLE_CUDA_KERNEL():
    
    # 首先我们调节 equalization 算法的参数
    # 调节时关闭 lsq_optimization 以缩短流程执行时间
    QS.equalization                         = True # 试试 True 或 False
    QS.equalization_setting.opt_level       = 1    # 试试 1 或 2
    QS.equalization_setting.iterations      = 10   # 试试 3, 10, 100
    QS.equalization_setting.value_threshold = 0.5  # 试试 0, 0.5, 2

    # 之后我们调节 LSQ 算法的参数,这是网络微调
    QS.lsq_optimization                            = True
    QS.lsq_optimization_setting.block_size         = 4       # 试试 1, 4, 6
    QS.lsq_optimization_setting.collecting_device  = 'cuda'  # 如果显存不够你就写 cpu
    QS.lsq_optimization_setting.is_scale_trainable = True    # 试试 True 或者 False
    QS.lsq_optimization_setting.lr                 = 1e-5    # 试试 1e-5, 3e-5, 1e-4
    QS.lsq_optimization_setting.steps              = 500     # 试试 300, 500, 2000

    quantized = quantize_torch_model(
        model=model, calib_dataloader=CALIBRATION, 
        calib_steps=32, input_shape=INPUT_SHAPE,
        collate_fn=collate_fn, platform=PLATFORM, setting=QS,
        onnx_export_file='Output/onnx.model', device=DEVICE, verbose=0)

    reports = graphwise_error_analyse(
        graph=quantized, running_device=DEVICE, 
        collate_fn=collate_fn, dataloader=CALIBRATION)

In [ ]:
# ------------------------------------------------------------
# Step - 6. 最后，我们导出模型，并在 onnxruntime 上完成校验
# ------------------------------------------------------------
ONNX_OUTPUT_PATH = 'Output/model.onnx'
executor, reference_outputs = TorchExecutor(quantized), []
for sample in CALIBRATION:
    reference_outputs.append(executor.forward(collate_fn(sample)))

export_ppq_graph(
    graph=quantized, platform=TargetPlatform.ONNXRUNTIME,
    graph_save_to=ONNX_OUTPUT_PATH)

In [ ]:
#比对结果
try:
    import onnxruntime
except ImportError as e:
    raise Exception('Onnxruntime is not installed.')

sess = onnxruntime.InferenceSession(ONNX_OUTPUT_PATH, providers=['CUDAExecutionProvider'])
onnxruntime_outputs = []
for sample in CALIBRATION:
    onnxruntime_outputs.append(sess.run(
        output_names=[name for name in quantized.outputs], 
        input_feed={'input.1': convert_any_to_numpy(sample)}))

name_of_output = [name for name in quantized.outputs]
for oidx, output in enumerate(name_of_output):
    y_pred, y_real = [], []
    for reference_output, onnxruntime_output in zip(reference_outputs, onnxruntime_outputs):
        y_pred.append(convert_any_to_torch_tensor(reference_output[oidx], device='cpu').unsqueeze(0))
        y_real.append(convert_any_to_torch_tensor(onnxruntime_output[oidx], device='cpu').unsqueeze(0))
    y_pred = torch.cat(y_pred, dim=0)
    y_real = torch.cat(y_real, dim=0)
    print(f'Simulating Error For {output}: {torch_snr_error(y_pred=y_pred, y_real=y_real).item() :.4f}')

##################################################################################################################

# 简单实现量化
    可用的都在api中，是函数和参数
    from ppq.api import load_calibration_dataset 给一些图片文件夹后形成数据集
    QuantizationSettingFactory里面包含了所有可以使用的参数

In [ ]:
from typing import Iterable
import torch
import torchvision
from ppq import BaseGraph, QuantizationSettingFactory, TargetPlatform
from ppq.api import export_ppq_graph, quantize_torch_model，load_onnx_graph
from torch.utils.data import DataLoader


#准备一个简单的数据集
BATCHSIZE = 32
INPUT_SHAPE = [3, 224, 224]
DEVICE = 'cuda' # only cuda is fully tested :(  For other executing device there might be bugs.
PLATFORM = TargetPlatform.PPL_DSP_INT8  # identify a target platform for your network.

def load_calibration_dataset() -> Iterable:
    return [torch.rand(size=INPUT_SHAPE) for _ in range(32)]

def collate_fn(batch: torch.Tensor) -> torch.Tensor:
    return batch.to(DEVICE)

# Load training data for creating a calibration dataloader.
calibration_dataset = load_calibration_dataset()
calibration_dataloader = DataLoader(
    dataset=calibration_dataset,
    batch_size=BATCHSIZE, shuffle=True)


# 准备模型
model = torchvision.models.mobilenet.mobilenet_v2(pretrained=True)
model = model.to(DEVICE)

# 量化参数
quant_setting = QuantizationSettingFactory.pplcuda_setting()
quant_setting.equalization = True # use layerwise equalization algorithm.
quant_setting.dispatcher   = 'conservative' # dispatch this network in conservertive way.


# 量化模型的唯一函数
quantized = quantize_torch_model(
    model=model,
    calib_dataloader=calibration_dataloader,
    calib_steps=32, 
    input_shape=[BATCHSIZE] + INPUT_SHAPE,
    setting=quant_setting,  
    collate_fn=collate_fn,
    platform=PLATFORM,
    onnx_export_file='Output/onnx.model',
    device=DEVICE,
    verbose=0)

# Quantization Result is a PPQ BaseGraph instance.
assert isinstance(quantized, BaseGraph)

# 导出模型
export_ppq_graph(graph=quantized, #模型
                 platform=PLATFORM,
                 graph_save_to='Output/quantized(onnx).onnx',
                 config_save_to='Output/quantized(onnx).json')

####################################################################################################################

# 细粒度操作

    导出器：每个平台都会对应一个导出器，实现export函数，生成onnx文件，它会调用内部的export_graph
    函数，所以也没什么好实现的，基本都是同一个


    1 定义自己的量化器，并完成注册
    量化器就是给每一个需要量化的算子，加上给定的量化参数
    量化器要对应平台，以平台来调用量化器。另外，平台还需要一个导出器，也需要去自定义实现
    案例在tutorial 的targetPlatform
    
    
    2 自定义PASS
    量化管道中的一个PASS是遍历所有的算子执行修改，可以自定义一个PASS对OP进行操作
    实际案例在tutorial 的fusion中
    
    
    
    
    
    

In [12]:
import torch
from torchvision import models

import ppq.lib as PFL
from ppq import TargetPlatform, TorchExecutor, graphwise_error_analyse
from ppq.api import ENABLE_CUDA_KERNEL, load_torch_model
from ppq.core.quant import (QuantizationPolicy, QuantizationProperty,
                            RoundingPolicy)
from ppq.quantization.optim import (LearnedStepSizePass, ParameterBakingPass,
                                    ParameterQuantizePass,
                                    RuntimeCalibrationPass)

# ------------------------------------------------------------
# 在这个例子中我们将向你展示如何使用 FP8 量化一个 Pytorch 模型
# 我们使用随机数据进行量化，这并不能得到好的量化结果。
# 在量化你的网络时，你应当使用真实数据和正确的预处理。
# ------------------------------------------------------------
model = models.efficientnet_b0(pretrained=True)
graph = load_torch_model(
    model, sample=torch.zeros(size=[1, 3, 224, 224]).cuda())
dataset = [torch.rand(size=[1, 3, 224, 224]) for _ in range(64)]

# -----------------------------------------------------------
# 我们将借助 PFL - PPQ Foundation Library, 即 PPQ 基础类库完成量化
# 这是 PPQ 自 0.6.6 以来推出的新的量化 api 接口，这一接口是提供给
# 算法工程师、部署工程师、以及芯片研发人员使用的，它更为灵活。
# 我们将手动使用 Quantizer 完成算子量化信息初始化, 并且手动完成模型的调度工作
# ------------------------------------------------------------
collate_fn  = lambda x: x.to('cuda')

# ------------------------------------------------------------
# 在开始之前，我需要向你介绍量化器、量化信息以及调度表
# 量化信息在 PPQ 中是由 TensorQuantizationConfig(TQC) 进行描述的
# 这个结构体描述了我要如何去量化一个数据，其中包含了量化位宽、量化策略、
# 量化 Scale, offset 等内容。
# ------------------------------------------------------------
from ppq import TensorQuantizationConfig as TQC

MyTQC = TQC(
    policy = QuantizationPolicy(
        QuantizationProperty.SYMMETRICAL + 
        QuantizationProperty.FLOATING +
        QuantizationProperty.PER_TENSOR + 
        QuantizationProperty.POWER_OF_2),
    rounding=RoundingPolicy.ROUND_HALF_EVEN,
    num_of_bits=8, quant_min=-448.0, quant_max=448.0, 
    exponent_bits=3, channel_axis=None,
    observer_algorithm='minmax'
)
# ------------------------------------------------------------
# 作为示例，我们创建了一个 "浮点" "对称" "Tensorwise" 的量化信息
# 这三者皆是该量化信息的 QuantizationPolicy 的一部分
# 同时要求该量化信息使用 ROUND_HALF_EVEN 方式进行取整
# 量化位宽为 8 bit，其中指数部分为 3 bit
# 量化上限为 448.0，下限则为 -448.0
# 这是一个 Tensorwise 的量化信息，因此 channel_axis = None
# observer_algorithm 表示在未来使用 minmax calibration 方法确定该量化信息的 scale

# 上述例子完成了该 TQC 的初始化，但并未真正启用该量化信息
# MyTQC.scale, MyTQC.offset 仍然为空，它们必须经过 calibration 才会具有有意义的值
# 并且他目前的状态 MyTQC.state 仍然是 Quantization.INITIAL，这意味着在计算时该 TQC 并不会参与运算。
# ------------------------------------------------------------

# ------------------------------------------------------------
# 接下来我们向你介绍量化器，这是 PPQ 中的一个核心类型
# 它的职责是为网络中所有处于量化区的算子初始化量化信息(TQC)
# PPQ 中实现了一堆不同的量化器，它们分别适配不同的情形
# 在这个例子中，我们分别创建了 TRT_INT8, GRAPHCORE_FP8, TRT_FP8 三种不同的量化器
# 由它们所生成的量化信息是不同的，为此你可以访问它们的源代码
# 位于 ppq.quantization.quantizer 中，查看它们初始化量化信息的逻辑。
# ------------------------------------------------------------
_ = PFL.Quantizer(platform=TargetPlatform.TRT_INT8, graph=graph)      # 取得 TRT_INT8 所对应的量化器
_ = PFL.Quantizer(platform=TargetPlatform.GRAPHCORE_FP8, graph=graph) # 取得 GRAPHCORE_FP8 所对应的量化器
quantizer = PFL.Quantizer(platform=TargetPlatform.TRT_FP8, graph=graph) # 取得 TRT_FP8 所对应的量化器

# ------------------------------------------------------------
# 调度器是 PPQ 中另一核心类型，它负责切分计算图
# 在量化开始之前，你的计算图将被切分成可量化区域，以及不可量化区域
# 不可量化区域往往就是那些执行 Shape 推断的算子所构成的子图
# *** 量化器只为量化区的算子初始化量化信息 ***
# 调度信息将被写在算子的属性中，你可以通过 op.platform 来访问每一个算子的调度信息
# ------------------------------------------------------------
dispatching = PFL.Dispatcher(graph=graph).dispatch(                       # 生成调度表
    quant_types=quantizer.quant_operation_types)

for op in graph.operations.values():
    # quantize_operation - 为算子初始化量化信息，platform 传递了算子的调度信息
    # 如果你的算子被调度到 TargetPlatform.FP32 上，则该算子不量化
    # 你可以手动修改调度信息
    dispatching['Op1'] = TargetPlatform.FP32        # 将 Op1 强行送往非量化区
    dispatching['Op2'] = TargetPlatform.UNSPECIFIED # 将 Op2 强行送往量化区
    
    # 你可能已经注意到了，我们并没有将 Op2 送往 TRT_FP8，而是将其送往 UNSPECIFIED 平台
    # 其含义是告诉量化器我们 "建议" 量化器对算子初始化量化信息，如果此时 Op2 不是量化器所支持的类型，则该算子仍然不会被量化
    # 但如果我们直接将 Op2 送往 TRT_FP8，不论如何该算子都将被量化
    
    quantizer.quantize_operation(
        op_name = op.name, platform = dispatching[op.name])

# ------------------------------------------------------------
# 在创建量化管线之前，我们需要初始化执行器，它用于模拟硬件并执行你的网络
# 请注意，执行器需要对网络结果进行分析并缓存分析结果，如果你的网络结构发生变化
# 你必须重新建立新的执行器。在上一步操作中，我们对算子进行了量化，这使得
# 普通的算子被量化算子替代，这一步操作将会改变网络结构。因此我们必须在其后建立执行器。
# ------------------------------------------------------------
executor = TorchExecutor(graph=graph, device='cuda')
executor.tracing_operation_meta(inputs=collate_fn(dataset[0]))
executor.load_graph(graph=graph)

# ------------------------------------------------------------
# 下面的过程将创建量化管线，它还是一个 PPQ 的核心类型
# 在 PPQ 中，模型的量化是由一个一个的量化过程(QuantizationOptimizationPass)完成的
# 量化管线 是 量化过程 的集合，在其中的量化过程将被逐个调用
# 从而实现对 TQC 中内容的修改，最终实现模型的量化
# 在这里我们为管线中添加了 4 个量化过程，分别处理不同的内容

# ParameterQuantizePass - 用于为模型中的所有参数执行 Calibration, 生成它们的 scale，并将对应 TQC 的状态调整为 ACTIVED
# RuntimeCalibrationPass - 用于为模型中的所有激活执行 Calibration, 生成它们的 scale，并将对应 TQC 的状态调整为 ACTIVED
# LearnedStepSizePass - 用于训练微调模型的权重，从而降低量化误差
# ParameterBakingPass - 用于执行模型参数烘焙

# 在 PPQ 中我们提供了数十种不同的 QuantizationOptimizationPass
# 你可以组合它们从而实现自定义的功能，也可以继承 QuantizationOptimizationPass 基类
# 从而创造出新的量化优化过程
#PASS是遍历图中的每一个变量进行处理
# ------------------------------------------------------------
pipeline = PFL.Pipeline([
    ParameterQuantizePass(),
    RuntimeCalibrationPass(),
    LearnedStepSizePass(
        steps=1000, is_scale_trainable=False, 
        lr=1e-4, block_size=4, collecting_device='cuda'),
    ParameterBakingPass()
])

with ENABLE_CUDA_KERNEL():
    # 调用管线完成量化
    pipeline.optimize(
        graph=graph, dataloader=dataset, verbose=True, 
        calib_steps=32, collate_fn=collate_fn, executor=executor)

    # 执行量化误差分析
    graphwise_error_analyse(
        graph=graph, running_device='cuda', 
        dataloader=dataset, collate_fn=collate_fn)

# ------------------------------------------------------------
# 在最后，我们导出计算图
# 同样地，我们根据不同推理框架的需要，写了一堆不同的网络导出逻辑
# 你通过参数 platform 告诉 PPQ 你的模型最终将部署在何处，
# PPQ 则会返回一个对应的 GraphExporter 对象，它将负责将 PPQ 的量化信息
# 翻译成推理框架所需的内容。你也可以自己写一个 GraphExporter 类并注册到 PPQ 框架中来。
# ------------------------------------------------------------
exporter = PFL.Exporter(platform=TargetPlatform.TRT_FP8)
exporter.export(file_path='Quantized.onnx', graph=graph)
